In [ ]:
# import first or else jupyter crashes

from desc.plotting import plot_section
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary,
)
import desc
import matplotlib.pyplot as plt


from desc.transform import Transform
from desc.equilibrium import Equilibrium
from desc.basis import ChebyshevZernikeBasis, chebyshev_z
from desc.compute import compute
from desc.grid import LinearGrid, ConcentricGrid, QuadratureGrid, Grid
from desc.compute.utils import get_transforms
from desc.objectives import (
    FixEndCapLambda,
    FixEndCapR,
    FixEndCapZ,
    FixBoundaryR,
    FixBoundaryZ,
    FixPsi,
    FixPressure,
    FixIota,
    FixAnisotropy,
    ForceBalance,
    ForceBalanceAnisotropic,
    ObjectiveFunction,
    CurrentDensity,
    MatchEndCapR,
    MatchEndCapZ,
    MatchEndCapLambda,
)
import numpy as np
from desc.optimize import Optimizer
from desc.profiles import PowerSeriesProfile
from desc.geometry import FourierRZToroidalSurface
from scipy.constants import mu_0
from mayavi import mlab


def chebygrid(N_grid):
    return np.concatenate(
        (
            [0],
            (-np.cos((2 * np.arange(N_grid) + 1) * np.pi / (2 * N_grid)) + 1) * np.pi,
            [2 * np.pi],
        )
    )


def grid_gen(L_grid, M_grid, N_grid, node_pattern="jacobi"):
    LMnodes = ConcentricGrid(L=L_grid, M=M_grid, N=0, node_pattern=node_pattern).nodes[
        :, :2
    ]
    Nnodes = chebygrid(N_grid)
    lm = np.tile(LMnodes, (Nnodes.size, 1))
    n = np.tile(Nnodes.reshape(-1, 1), (1, LMnodes.shape[0])).reshape(-1, 1)
    nodes = np.concatenate((lm, n), axis=1)

    # RG: weights and spacing defined here
    # just for the sake of compilation. Must be checked
    weights = np.ones(nodes.shape[0])
    spacing = np.ones_like(nodes)

    spacing[1:, 1] = np.diff(nodes[:, 1])
    spacing[1:, 2] = np.diff(nodes[:, 2])

    return Grid(nodes, spacing=spacing, weights=weights)


def get_lm_mode(basis, coeff, zeta, L, M, func_zeta=chebyshev_z):
    modes = basis.modes
    lm = 0
    for i, (l, m, n) in enumerate(modes):
        if l == L and m == M:
            lm += func_zeta(zeta, n) * coeff[i]
    return lm


def chebygrid(N_grid):
    return np.concatenate(
        (
            [0],
            (-np.cos((2 * np.arange(N_grid) + 1) * np.pi / (2 * N_grid)) + 1) * np.pi,
            [2 * np.pi],
        )
    )


def grid_gen(L_grid, M_grid, N_grid, node_pattern="jacobi"):
    LMnodes = ConcentricGrid(L=L_grid, M=M_grid, N=0, node_pattern=node_pattern).nodes[
        :, :2
    ]
    Nnodes = chebygrid(N_grid)
    lm = np.tile(LMnodes, (Nnodes.size, 1))
    n = np.tile(Nnodes.reshape(-1, 1), (1, LMnodes.shape[0])).reshape(-1, 1)
    nodes = np.concatenate((lm, n), axis=1)

    # RG: weights and spacing defined here
    # just for the sake of compilation. Must be checked
    weights = np.ones(nodes.shape[0])
    spacing = np.ones_like(nodes)

    spacing[1:, 1] = np.diff(nodes[:, 1])
    spacing[1:, 2] = np.diff(nodes[:, 2])

    return Grid(nodes, spacing=spacing, weights=weights)


def get_lm_mode(basis, coeff, zeta, L, M, func_zeta=chebyshev_z):
    modes = basis.modes
    lm = 0
    for i, (l, m, n) in enumerate(modes):
        if l == L and m == M:
            lm += func_zeta(zeta, n) * coeff[i]
    return lm


plt.rcParams.update(
    {
        "font.size": 16,  # Default text size
        "axes.titlesize": 18,  # Title font
        "axes.labelsize": 16,  # x/y label font
        "xtick.labelsize": 14,  # Tick font
        "ytick.labelsize": 14,
        "legend.fontsize": 14,
    }
)


## test
surf = FourierRZToroidalSurface(
    R_lmn=[10, 1, -0.5],
    modes_R=[[0, 0], [1, 0], [1, 2]],
    Z_lmn=[0, -1, 0.5],
    modes_Z=[[0, 0], [-1, 0], [-1, 2]],
    NFP=1,
    sym=False,
    mirror=True,
)

iota = PowerSeriesProfile(params=[0.0, 0])

p = PowerSeriesProfile(params=[0.001 / mu_0, -0.001 / mu_0], modes=[0, 2])
p.set_params(0, a=-1e-3)  # = array([-1000])
p.set_params(2, a=1e-3)  # = array([-1000])

In [ ]:
from desc.objectives._equilibrium import (
    ForceBalanceAnisotropic_3D,
    B_on_Axis,
    ForceBalanceAnisotropic_3D,
    ForceBalanceAnisotropic_3D_explicit,
)

In [ ]:
import jax.numpy as jnp

In [ ]:
#### Profile must be a FourierZernikeProfile
from desc.profiles import FourierZernikeProfile
from desc.grid import Grid

In [ ]:
# Make full 3D grid to define your data
rho_vals = np.linspace(0, 1, 32, endpoint=True)
theta_vals = np.linspace(0, 2 * np.pi, 16, endpoint=False)
zeta_vals = np.linspace(0, 2 * np.pi, 34, endpoint=True)

rho, theta, zeta = np.meshgrid(rho_vals, theta_vals, zeta_vals, indexing="ij")

# Define pressure as a function of rho and zeta only
p_perp_vals = 5 * (
    10 * (1.00 - rho) * (np.abs(zeta - np.pi) / np.pi) ** 2 + 10 * (1 - rho)
)

In [ ]:
plt.plot(p_perp_vals[0, 0, :])
plt.plot(p_perp_vals[15, 0, :])
plt.plot(p_perp_vals[-1, 0, :])

In [ ]:
# Flatten for fitting
p_perp_profile = FourierZernikeProfile.from_values(
    rho.ravel(),
    theta.ravel(),
    zeta.ravel(),
    p_perp_vals.ravel(),
    L=20,
    M=0,
    N=20,
    NFP=1,
)

In [ ]:
nodes = np.stack([rho.ravel(), theta.ravel(), zeta.ravel()], axis=1)
grid = Grid(nodes)
p_fit = p_perp_profile.compute(grid)
plt.plot((p_perp_vals.ravel() - p_fit) / p_perp_vals.ravel(), ".")

In [ ]:
# Flatten for fitting
empty_profile = FourierZernikeProfile.from_values(
    rho.ravel(),
    theta.ravel(),
    zeta.ravel(),
    p_perp_vals.ravel() * 0,
    L=20,
    M=0,
    N=20,
    NFP=1,
)

In [ ]:
# test

surf = FourierRZToroidalSurface(
    R_lmn=[10, 1, -0.5],
    modes_R=[[0, 0], [1, 0], [1, 2]],
    Z_lmn=[0, -1, 0.5],
    modes_Z=[[0, 0], [-1, 0], [-1, 2]],
    NFP=1,
    sym=False,
    mirror=True,
)

In [ ]:
surf_init = surf.copy()

In [ ]:
eq = Equilibrium(
    anisotropy=empty_profile,
    surface=surf_init,
    L=5,
    M=2,
    N=8,
    mirror=True,
    pressure=p_perp_profile,
    iota=iota,
    sym=False,
)

In [ ]:
eq.Psi = 1

In [ ]:
# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
surface = 0
zeta_axis = np.linspace(0, 2 * np.pi, 100)
rho_axis = np.ones_like(zeta_axis) * surface
theta_axis = np.zeros_like(zeta_axis)
nodes = np.stack([rho_axis, theta_axis, zeta_axis], axis=1)
grid = Grid(nodes, spacing=np.ones_like(nodes), weights=np.ones_like(zeta_axis))

In [ ]:
p_eq = eq.compute(["p_perp", "p_parallel", "|B|"], grid=grid)

plt.figure()
plt.plot(p_eq["p_perp"], label="eq.compute['p_perp']")
plt.plot(p_eq["p_parallel"], label=r"eq.compute['p_parallel']")

plt.xlabel("zeta")
plt.ylabel("p")
plt.legend()
plt.title(r"$\rho$ = {}, $\theta$ = 0 ".format(surface))

In [ ]:
# Compute on axis
data_axis = eq.compute(["|B|"], grid=grid)
B_axis = data_axis["|B|"]

plt.figure()
plt.plot(zeta_axis, B_axis)
plt.title("MR = {}".format(np.max(B_axis) / np.min(B_axis)))

In [ ]:
grid = grid_gen(20, 20, 20)
fb = ForceBalanceAnisotropic(eq=eq, grid=grid)
fb3 = ForceBalanceAnisotropic_3D(eq=eq, grid=grid)
fb_e = ForceBalanceAnisotropic_3D_explicit(eq=eq, grid=grid)
on_axis = B_on_Axis(eq=eq, monotonic=True, MR=True, MR_target=10, weight=1e10)

In [ ]:
fb.build()
fb3.build()
fb_e.build()
on_axis.build()

In [ ]:
plt.plot(fb.compute(eq.params_dict))

In [ ]:
plt.plot(fb3.compute(eq.params_dict))

In [ ]:
plt.plot(fb_e.compute(eq.params_dict))

In [ ]:
plt.plot(on_axis.compute(eq.params_dict))

In [ ]:
constraints = (
    # FixEndCapLambda(0, eq=eq),
    # FixEndCapR(0, eq=eq),
    # FixEndCapZ(0, eq=eq),
    # FixEndCapLambda(2*np.pi, eq=eq),
    # FixEndCapR(2*np.pi, eq=eq),
    # FixEndCapZ(2*np.pi, eq=eq),
    MatchEndCapR(eq=eq),
    MatchEndCapZ(eq=eq),
    MatchEndCapLambda(eq=eq),
    # MatchMidplaneLambda(eq=eq),
    # MatchMidplaneR(eq=eq),
    # MatchMidplaneZ(eq=eq),
    FixBoundaryR(eq=eq),
    FixBoundaryZ(eq=eq),
    FixPressure(eq=eq),
    FixIota(eq=eq),
    FixPsi(eq=eq),
    FixAnisotropy(eq=eq),
)
optimizer = Optimizer("lsq-exact")
grid = grid_gen(20, 20, 20)

objective1 = ForceBalanceAnisotropic_3D_explicit(eq=eq, grid=grid)
# objective2 = ParallelCurrentConstraint3D(eq=eq, grid=grid, weight = 10)
objective3 = B_on_Axis(eq=eq, Nz=100, monotonic=True, MR=True, MR_target=10, weight=1e5)
objectives = [objective1]
obj = ObjectiveFunction(objectives=objective1)

eq.solve(
    objective=obj,
    constraints=constraints,
    optimizer=optimizer,
    ftol=1e-6,
    xtol=1e-16,
    gtol=1e-16,
    maxiter=25,
    verbose=3,
)

In [ ]:
f0 = (
    eq.compute("<|F|>_vol")["<|F|>_vol"]
    / eq.compute("<|grad(|B|^2)|/2mu0>_vol")["<|grad(|B|^2)|/2mu0>_vol"]
)


print(f"Force error after solve: {f0:.4e}")

In [ ]:
fba_t = ForceBalanceAnisotropic_3D_explicit(eq=eq, grid=grid)
fba_t.build()
fba_test = fba_t.compute(eq.params_dict)

plt.figure()
plt.plot(fba_test)

In [ ]:
fba_t = ForceBalanceAnisotropic_3D(eq=eq, grid=grid)
fba_t.build()
fba_test = fba_t.compute(eq.params_dict)

plt.figure()
plt.plot(fba_test)

In [ ]:
fba_t = ParallelCurrentConstraint3D(eq=eq, grid=grid, weight=10)
fba_t.build()
fba_test = fba_t.compute(eq.params_dict)

plt.figure()
plt.plot(fba_test)

In [ ]:
fba_t = B_on_Axis(eq=eq, monotonic=True, MR=True, MR_target=10)
fba_t.build()
fba_test = fba_t.compute(eq.params_dict)

plt.figure()
plt.plot(fba_test)

In [ ]:
fba_t = ForceBalanceAnisotropic_3D(eq=eq, grid=grid)
fba_t.build()
fba_test = fba_t.compute(eq.params_dict)

plt.figure()
plt.plot(fba_test)

In [ ]:
### plotting tools

In [ ]:
### define a grid

In [ ]:
grid = grid_gen(2, 10, 40)

In [ ]:
data = eq.compute(
    [
        "X",
        "Y",
        "Z",
        "|B|",
        "<beta>_vol",
        "p_parallel",
        "|F|",
        "p_perp",
        "grad(p)",
        "J",
        "|J|",
        "J_rho",
        "J_theta",
        "J_zeta",
        "J*B",
        "J_parallel",
    ],
    grid=grid,
)

In [ ]:
# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
Nz = 100
zeta_axis = np.linspace(0, 2 * np.pi, Nz)
rho_axis = np.ones_like(zeta_axis) * 0
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p_perp", "p_parallel", "Y", "beta_a"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_perp_axis = data_axis["p_perp"]
p_par_axis = data_axis["p_parallel"]

plt.figure()
plt.plot(zeta_axis, B_axis, label=r"$|B|$")
plt.title("MR = {:.2f}".format(np.max(B_axis) / np.min(B_axis)))
plt.legend()

In [ ]:
plt.figure()
plt.plot(data_axis["Y"], p_perp_axis, label=r"$p_\perp(z)$")


plt.plot(data_axis["Y"], p_par_axis, label=r"$p_\parallel(z)$")
plt.legend()

In [ ]:
# data = eq.compute(["F_rho_p_perp","F_theta_p_perp","F_zeta_p_perp","|grad(theta)|","|grad(rho)|","|grad(zeta)|"])

In [ ]:
### LCFS metrics ? -- ordered set of points

In [ ]:
# %% Get fieldlines on the LCFS
# assert isinstance(eq, Equilibrium)

Nz = 200
Nt = 200
rho_surf = 0.1

theta = np.linspace(0, 2 * np.pi, Nt)
zeta = np.linspace(0, 2 * np.pi, Nz, endpoint=True)

theta_mesh, zeta_mesh = np.meshgrid(theta, zeta, indexing="ij")
shape = theta_mesh.shape
coords_sfl = np.stack(
    [
        np.ones_like(theta_mesh.reshape(-1)) * rho_surf,
        theta_mesh.reshape(-1),
        zeta_mesh.reshape(-1),
    ],
    axis=-1,
)

coords_geo = eq.compute_theta_coords(coords_sfl)
coords_data = eq.compute(
    ["R", "Z", "zeta", "p", "|B|"], grid=Grid(nodes=coords_geo, sort=False)
)


lines = np.array([coords_data["R"], coords_data["zeta"], coords_data["Z"]])
lines = lines.reshape(3, shape[0], shape[1])
lines = np.moveaxis(lines, [0, 1], [1, 0])

B_on_lines = coords_data["|B|"]
B_on_lines = B_on_lines.reshape(shape[0], shape[1])

In [ ]:
data_mesh = eq.compute(
    [
        "beta_a",
        "theta",
        "X",
        "Y",
        "Z",
        "|B|",
        "B",
        "p_parallel",
        "p_perp",
        "J",
        "e^theta",
        "e^rho",
        "e^zeta",
        "grad(alpha)",
    ],
    grid=Grid(nodes=coords_geo, sort=False),
)
p_par_mesh = (
    data_mesh["p_perp"]
    + data_mesh["beta_a"] * data_mesh["|B|"] * data_mesh["|B|"] / mu0
)

### axial pressure profile
mu0 = 4 * np.pi * 1e-7

# ---- Grid for axis (rho = 0, theta = 0, zeta from 0 to 2pi) ----
zeta_axis = np.linspace(0, 2 * np.pi, 100)
rho_axis = np.zeros_like(zeta_axis)
theta_axis = np.zeros_like(zeta_axis)
grid_axis = Grid(np.stack([rho_axis, theta_axis, zeta_axis], axis=1))

# Compute on axis
data_axis = eq.compute(["|B|", "p_perp", "p_parallel", "beta_a"], grid=grid_axis)
B_axis = data_axis["|B|"]
p_perp_axis = data_axis["p_perp"]
p_par_axis = p_perp_axis + data_axis["beta_a"] * B_axis * B_axis / mu0


# ---- Grid for midplane slice (zeta = pi, theta = 0, rho from 0 to 1) ----
rho_mid = np.linspace(0, 1, 100)
theta_mid = np.zeros_like(rho_mid)
zeta_mid = np.ones_like(rho_mid) * np.pi
grid_mid = Grid(np.stack([rho_mid, theta_mid, zeta_mid], axis=1))

# Compute on midplane profile
data_mid = eq.compute(["p_perp", "|B|", "p_parallel", "beta_a"], grid=grid_mid)
B_mid = data_mid["|B|"]
p_perp_mid = data_mid["p_perp"]
p_par_mid = p_perp_mid + data_mid["beta_a"] * B_mid * B_mid / mu0


X2d = np.reshape(data_mesh["X"], (Nz, Nt))
Y2d = np.reshape(data_mesh["Y"], (Nz, Nt))
Z2d = np.reshape(data_mesh["Z"], (Nz, Nt))
B2d = np.reshape(data_mesh["|B|"], (Nz, Nt))
J2d = np.reshape(data_mesh["J"], (Nz, Nt, 3))
theta2d = np.reshape(data_mesh["theta"], (Nz, Nt))

etheta = np.reshape(data_mesh["e^theta"], (Nz, Nt, 3))
ezeta = np.reshape(data_mesh["e^zeta"], (Nz, Nt, 3))
erho = np.reshape(data_mesh["e^rho"], (Nz, Nt, 3))

e_theta_n = etheta / np.linalg.norm(etheta, axis=-1)[:, :, None]
e_zeta_n = ezeta / np.linalg.norm(ezeta, axis=-1)[:, :, None]
e_rho_n = erho / np.linalg.norm(erho, axis=-1)[:, :, None]

J2d_n = J2d / np.linalg.norm(J2d, axis=-1)[:, :, None]

comp_theta = np.sum(J2d_n * e_theta_n, axis=-1)
comp_zeta = np.sum(J2d_n * e_zeta_n, axis=-1)
comp_rho = np.sum(J2d_n * e_rho_n, axis=-1)

Bv = np.reshape(data_mesh["B"], (Nz, Nt, 3))
B2d_n = Bv / np.linalg.norm(Bv, axis=-1)[:, :, None]

JdotB = np.sum(J2d_n * B2d_n, axis=-1)

# gradp = np.reshape(data_mesh['grad(p)'],(Nz,Nt,3))
# gradp_n =  gradp / np.linalg.norm(gradp,axis = -1)[:,:,None]

# Jdotgradp = np.sum(J2d_n*gradp_n,axis = -1)

# Bdotgradp = np.sum(B2d_n*gradp_n,axis = -1)

JdotB2d_n = np.sum(J2d_n * B2d_n, axis=-1)

gradalpha = np.reshape(data_mesh["grad(alpha)"], (Nz, Nt, 3))
gradalpha_n = gradalpha / np.linalg.norm(gradalpha, axis=-1)[:, :, None]

Jdotgradalpha_n = np.sum(J2d_n * gradalpha_n, axis=-1)

beta = 2 * (data_mesh["p_perp"] + p_par_mesh) * mu_0 / (data_mesh["|B|"] ** 2 + 1e-20)
beta_axis = (
    2 * (data_axis["p_perp"] + p_par_axis) * mu_0 / (data_axis["|B|"] ** 2 + 1e-20)
)
beta_midplane = (
    2 * (data_mid["p_perp"] + p_par_mid) * mu_0 / (data_mid["|B|"] ** 2 + 1e-20)
)


beta2d = np.reshape(beta, (Nz, Nt))

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(16, 11))

nlevels = 500
cmap = "coolwarm"

# field lines
i = 0
for line in lines[::10]:
    fieldlines = axs[0, 0].scatter(line[1], line[2], c=B_on_lines[i], s=0.2)
    i = i + 10
fig.colorbar(fieldlines, ax=axs[0, 0], label=r"|B|")

#

axs[0, 1].plot(zeta_axis, p_par_axis, label=r"$p_{\parallel}$")
axs[0, 1].plot(zeta_axis, p_perp_axis, label=r"$p_{\perp}$")
axs[0, 1].legend()

axs[0, 2].plot(rho_mid, p_par_mid, label=r"$p_{\parallel}$")
axs[0, 2].plot(rho_mid, p_perp_mid, label=r"$p_{\perp}$")
axs[0, 2].legend()


im = axs[1, 0].contour(Y2d, theta2d, beta2d, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[1, 0])


im = axs[1, 1].plot(zeta_axis, beta_axis)
im = axs[1, 2].plot(rho_mid, beta_midplane)


# J
im = axs[2, 1].contour(Y2d, theta2d, comp_rho, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 1])

im = axs[2, 0].contour(Y2d, theta2d, Jdotgradalpha_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 0])

im = axs[2, 2].contour(Y2d, theta2d, JdotB2d_n, levels=nlevels, cmap=cmap)
fig.colorbar(im, ax=axs[2, 2])


# titles
axs[0, 0].set_title(r"$\rho = {}$ Field lines".format(rho_surf))
axs[0, 1].set_title("axial profile")
axs[0, 2].set_title("radial profile (midplane)")

axs[1, 0].set_title(r"$\beta$ at $\rho = {}$ ".format(rho_surf))
axs[1, 1].set_title(r"axial $\beta$")
axs[1, 2].set_title(r"radial $\beta$")

axs[2, 2].set_title(r"$J \cdot b$")
axs[2, 0].set_title(r"$J \cdot \nabla \alpha$")
axs[2, 1].set_title(r"$J \cdot e^\rho$")

axs[0, 0].set_ylabel(r"$Z$")
axs[0, 1].set_ylabel(r"$pressure(Pa)$")
axs[0, 2].set_ylabel(r"$pressure(Pa)$")

axs[2, 2].set_ylabel(r"$\theta$")
axs[2, 0].set_ylabel(r"$\theta$")
axs[2, 1].set_ylabel(r"$\theta$")


axs[2, 0].set_xlabel(r"$Y$")
axs[2, 1].set_xlabel(r"$Y$")
axs[2, 2].set_xlabel(r"$Y$")


axs[0, 0].grid()
axs[1, 1].grid()
axs[1, 2].grid()

axs[2, 2].grid()
axs[2, 0].grid()
axs[2, 1].grid()

fig.tight_layout()

In [ ]:
# grid = grid_gen(1, 20, 80)

In [ ]:
# %% Get fieldlines on the LCFS
# assert isinstance(eq, Equilibrium)

Nz = 50
Nt = 50
Nr = 2

theta = np.linspace(0, 2 * np.pi, Nt)
zeta = np.linspace(0, 2 * np.pi, Nz, endpoint=True)
rho = np.linspace(0, 1, Nr)

full_coords = []
for r in rho:

    theta_mesh, zeta_mesh = np.meshgrid(theta, zeta, indexing="ij")
    shape = theta_mesh.shape
    coords_sfl = np.stack(
        [
            np.ones_like(theta_mesh.reshape(-1)) * r,
            theta_mesh.reshape(-1),
            zeta_mesh.reshape(-1),
        ],
        axis=-1,
    )

    coords_geo = eq.compute_theta_coords(coords_sfl)

    full_coords.append(coords_geo)

full_coords = np.array(full_coords)

In [ ]:
coords_geo = np.array(
    [
        full_coords[:, :, 0].ravel(),
        full_coords[:, :, 1].ravel(),
        full_coords[:, :, 2].ravel(),
    ]
).T

In [ ]:
grid = Grid(nodes=coords_geo, sort=False)

In [ ]:
data = eq.compute(
    [
        "isodynamicity",
        "X",
        "Y",
        "Z",
        "|B|",
        "<beta>_vol",
        "beta_a",
        "|F|",
        "p",
        "grad(p)",
        "J",
        "|J|",
        "J_rho",
        "J_theta",
        "J_zeta",
        "J*B",
        "J_parallel",
    ],
    grid=grid,
)

In [ ]:
data["<beta>_vol"]

In [ ]:
# 3D

x = data["X"]
y = data["Y"]
z = data["Z"]
b = data["|B|"]
b = data["isodynamicity"]

fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0.0, 0.0, 0.0))


points = mlab.points3d(x, y, z, b, scale_factor=0.01, scale_mode="none")
# points = mlab.points3d(x,y,z,scale_factor = .05, scale_mode = 'none')

mlab.colorbar()


mlab.show()

In [ ]:
# 3D

x = data["X"]
y = data["Y"]
z = data["Z"]
# b = data['p']
# bv = data['grad(p)']
bv = data["B"]

fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0.0, 0.0, 0.0))

points = mlab.points3d(x, y, z, scale_factor=0.01, scale_mode="none")
arrows = mlab.quiver3d(
    x,
    y,
    z,
    bv[:, 0],
    bv[:, 1],
    bv[:, 2],
    scale_factor=0.1,
    line_width=10,
    scale_mode="none",
)

bv = data["J"]
arrows1 = mlab.quiver3d(
    x,
    y,
    z,
    bv[:, 0],
    bv[:, 1],
    bv[:, 2],
    scale_factor=0.1,
    line_width=10,
    scale_mode="none",
    color=(1, 0, 1),
)

# bv = data['e^zeta']
# arrows2 = mlab.quiver3d(x,y,z,bv[:,0],bv[:,1],bv[:,2], scale_factor=.1, line_width=10,scale_mode = 'none',color = (1,0,0))

# bv = data['e^theta']
# arrows3 = mlab.quiver3d(x,y,z,bv[:,0],bv[:,1],bv[:,2], scale_factor=.1, line_width=10,scale_mode = 'none',color = (0,0,1))


# mlab.colorbar(arrows)


mlab.show()

In [ ]:
# --- Extract quantities ---
p = data["p"]  # pressure

B_mag = data["|B|"]  # magnitude of magnetic field
beta = (2 * mu_0 * p) / (B_mag**2 + 1e-20)  # plasma beta with small epsilon for safety

In [ ]:
beta = data["beta_a"] + 2 * data["p"] * mu_0 / (
    data["|B|"] ** 2 + 1e-20
)  # plasma beta with small epsilon for safety

In [ ]:
p_par = data["p"] + (data["|B|"] ** 2 / mu0) * data["beta_a"]

In [ ]:
beta = (2 * mu_0 * (data["p"] + p_par)) / (B_mag**2 + 1e-20)

In [ ]:
plt.plot(beta, ".")

In [ ]:
# 3D

x = data["X"]
y = data["Y"]
z = data["Z"]
b = beta

fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0.0, 0.0, 0.0))


points = mlab.points3d(x, y, z, b, scale_factor=0.05, scale_mode="none")
# points = mlab.points3d(x[filter],y[filter],z[filter],b[filter],scale_factor = .01, scale_mode = 'none')

mlab.colorbar()


mlab.show()